# Port Congestion Prediction Model Training

This notebook trains a LightGBM model to predict port waiting times at bulk carrier discharge ports.

**Target Ports:**
- China: Qingdao, Rizhao, Caofeidian, Fangcheng
- India: Mundra, Vizag

**Target Metrics:**
- MAE < 1.5 days
- RMSE < 2.0 days
- % within 1 day of actual > 60%
- % within 2 days of actual > 80%

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# ML imports
import lightgbm as lgb
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
import joblib

# Local imports
from ml_models.feature_engineering import FeatureEngineer
from ml_models.holiday_calendar import HolidayCalendar

print("Imports successful!")

## 1. Data Loading

Load the 5M row daily port activity data and filter to target ports.

In [ ]:
# Load data
print("Loading port activity data...")
activity_df = pd.read_csv('Daily_Port_Activity_Data_and_Trade_Estimates.csv')
print(f"Loaded {len(activity_df):,} rows")

# Load port database for capacity info
ports_df = pd.read_csv('PortWatch_ports_database.csv')
print(f"Loaded {len(ports_df):,} ports")

In [ ]:
# Target port IDs
TARGET_PORTS = {
    'port1069': 'Qingdao',
    'port1105': 'Rizhao',
    'port339': 'Fangcheng',
    'port1266': 'Caofeidian',
    'port777': 'Mundra',
    'port1367': 'Vizag',
}

# Filter to target ports
target_df = activity_df[activity_df['portid'].isin(TARGET_PORTS.keys())].copy()
target_df['date'] = pd.to_datetime(target_df['date'])
target_df = target_df.sort_values(['portid', 'date'])

print(f"Filtered to {len(target_df):,} rows for target ports")
print(f"Date range: {target_df['date'].min()} to {target_df['date'].max()}")

In [ ]:
# Show data per port
port_counts = target_df.groupby('portid').size().reset_index(name='count')
port_counts['port_name'] = port_counts['portid'].map(TARGET_PORTS)
print("\nRows per port:")
print(port_counts)

## 2. Exploratory Data Analysis

In [ ]:
# Plot port calls distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (port_id, port_name) in enumerate(TARGET_PORTS.items()):
    port_data = target_df[target_df['portid'] == port_id]
    axes[idx].hist(port_data['portcalls_dry_bulk'], bins=50, alpha=0.7)
    axes[idx].set_title(f'{port_name} - Port Calls Distribution')
    axes[idx].set_xlabel('Daily Port Calls')
    axes[idx].set_ylabel('Frequency')

plt.tight_layout()
plt.show()

In [ ]:
# Time series plot
fig, axes = plt.subplots(3, 2, figsize=(15, 12))
axes = axes.flatten()

for idx, (port_id, port_name) in enumerate(TARGET_PORTS.items()):
    port_data = target_df[target_df['portid'] == port_id].set_index('date')
    # Rolling 7-day average
    rolling = port_data['portcalls_dry_bulk'].rolling(7).mean()
    axes[idx].plot(rolling, alpha=0.8)
    axes[idx].set_title(f'{port_name} - 7-Day Rolling Port Calls')
    axes[idx].set_xlabel('Date')
    axes[idx].set_ylabel('Port Calls (7-day avg)')

plt.tight_layout()
plt.show()

In [ ]:
# Seasonality analysis - Monthly patterns
target_df['month'] = target_df['date'].dt.month

monthly_avg = target_df.groupby(['portid', 'month'])['portcalls_dry_bulk'].mean().reset_index()
monthly_avg['port_name'] = monthly_avg['portid'].map(TARGET_PORTS)

fig, ax = plt.subplots(figsize=(12, 6))
for port_id, port_name in TARGET_PORTS.items():
    port_monthly = monthly_avg[monthly_avg['portid'] == port_id]
    ax.plot(port_monthly['month'], port_monthly['portcalls_dry_bulk'], marker='o', label=port_name)

ax.set_xlabel('Month')
ax.set_ylabel('Average Daily Port Calls')
ax.set_title('Seasonal Patterns by Port')
ax.legend()
ax.set_xticks(range(1, 13))
ax.set_xticklabels(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])
plt.tight_layout()
plt.show()

## 3. Target Variable Creation

Create the congestion proxy (delay_days) target variable.

In [ ]:
# Initialize feature engineer
feature_engineer = FeatureEngineer('PortWatch_ports_database.csv')

# Create target variable for each port
all_data = []

for port_id in TARGET_PORTS.keys():
    port_data = target_df[target_df['portid'] == port_id].copy()
    port_data = port_data.sort_values('date')
    
    # Create delay_days target
    port_data['delay_days'] = feature_engineer.create_target_variable(port_data, port_id)
    all_data.append(port_data)
    
    print(f"{TARGET_PORTS[port_id]}: mean delay = {port_data['delay_days'].mean():.2f} days")

combined_df = pd.concat(all_data, ignore_index=True)

In [ ]:
# Plot target variable distribution
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for idx, (port_id, port_name) in enumerate(TARGET_PORTS.items()):
    port_data = combined_df[combined_df['portid'] == port_id]
    axes[idx].hist(port_data['delay_days'], bins=30, alpha=0.7, color='coral')
    axes[idx].axvline(port_data['delay_days'].mean(), color='red', linestyle='--', label='Mean')
    axes[idx].set_title(f'{port_name} - Delay Distribution')
    axes[idx].set_xlabel('Delay (days)')
    axes[idx].set_ylabel('Frequency')
    axes[idx].legend()

plt.tight_layout()
plt.show()

## 4. Feature Engineering

In [ ]:
# Engineer features for all ports
all_features = []

for port_id in TARGET_PORTS.keys():
    port_data = combined_df[combined_df['portid'] == port_id].copy()
    port_data = port_data.sort_values('date')
    
    # Full feature engineering
    features_df = feature_engineer.engineer_features(port_data, port_id, include_target=True)
    all_features.append(features_df)
    
    print(f"{TARGET_PORTS[port_id]}: {len(features_df)} rows, {len(features_df.columns)} features")

training_df = pd.concat(all_features, ignore_index=True)
print(f"\nTotal training data: {len(training_df):,} rows")

In [ ]:
# Show feature columns
print("Feature columns:")
print(training_df.columns.tolist())

In [ ]:
# Drop rows with NaN from lag features
feature_cols = feature_engineer.get_feature_columns()
available_features = [c for c in feature_cols if c in training_df.columns]

print(f"Using {len(available_features)} features")
print(available_features)

## 5. Model Training

In [ ]:
# Prepare train/validation/test splits (time-based)
training_df['date'] = pd.to_datetime(training_df['date'])

# Train: 2019-01-01 to 2023-12-31
# Validation: 2024-01-01 to 2024-06-30
# Test: 2024-07-01 to latest

train_mask = training_df['date'] < '2024-01-01'
val_mask = (training_df['date'] >= '2024-01-01') & (training_df['date'] < '2024-07-01')
test_mask = training_df['date'] >= '2024-07-01'

train_df = training_df[train_mask].dropna(subset=['delay_days'])
val_df = training_df[val_mask].dropna(subset=['delay_days'])
test_df = training_df[test_mask].dropna(subset=['delay_days'])

print(f"Train: {len(train_df):,} rows ({train_df['date'].min()} to {train_df['date'].max()})")
print(f"Val:   {len(val_df):,} rows ({val_df['date'].min()} to {val_df['date'].max()})")
print(f"Test:  {len(test_df):,} rows ({test_df['date'].min()} to {test_df['date'].max()})")

In [ ]:
# Prepare feature matrices
X_train = train_df[available_features].fillna(0)
y_train = train_df['delay_days']

X_val = val_df[available_features].fillna(0)
y_val = val_df['delay_days']

X_test = test_df[available_features].fillna(0)
y_test = test_df['delay_days']

print(f"X_train shape: {X_train.shape}")
print(f"X_val shape: {X_val.shape}")
print(f"X_test shape: {X_test.shape}")

In [ ]:
# LightGBM parameters
params = {
    'objective': 'regression',
    'metric': ['mae', 'rmse'],
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'min_data_in_leaf': 100,
    'verbose': -1,
    'seed': 42,
}

# Create datasets
train_data = lgb.Dataset(X_train, label=y_train)
val_data = lgb.Dataset(X_val, label=y_val, reference=train_data)

# Train model
print("Training LightGBM model...")
model = lgb.train(
    params,
    train_data,
    num_boost_round=1000,
    valid_sets=[train_data, val_data],
    valid_names=['train', 'val'],
    callbacks=[
        lgb.early_stopping(stopping_rounds=50),
        lgb.log_evaluation(period=100)
    ]
)

print(f"\nBest iteration: {model.best_iteration}")

## 6. Model Evaluation

In [ ]:
# Make predictions
y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

# Calculate metrics
def calculate_metrics(y_true, y_pred, name):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    within_1_day = np.mean(np.abs(y_true - y_pred) <= 1) * 100
    within_2_days = np.mean(np.abs(y_true - y_pred) <= 2) * 100
    
    print(f"\n{name} Metrics:")
    print(f"  MAE: {mae:.3f} days")
    print(f"  RMSE: {rmse:.3f} days")
    print(f"  Within 1 day: {within_1_day:.1f}%")
    print(f"  Within 2 days: {within_2_days:.1f}%")
    
    return {'mae': mae, 'rmse': rmse, 'within_1_day': within_1_day, 'within_2_days': within_2_days}

train_metrics = calculate_metrics(y_train, y_train_pred, "Training")
val_metrics = calculate_metrics(y_val, y_val_pred, "Validation")
test_metrics = calculate_metrics(y_test, y_test_pred, "Test")

In [ ]:
# Check if we meet target metrics
print("\n" + "="*50)
print("TARGET METRICS CHECK (Test Set):")
print("="*50)

checks = [
    ("MAE < 1.5 days", test_metrics['mae'] < 1.5),
    ("RMSE < 2.0 days", test_metrics['rmse'] < 2.0),
    ("Within 1 day > 60%", test_metrics['within_1_day'] > 60),
    ("Within 2 days > 80%", test_metrics['within_2_days'] > 80),
]

for check_name, passed in checks:
    status = "PASS" if passed else "FAIL"
    print(f"  [{status}] {check_name}")

In [ ]:
# Per-port metrics
print("\nPer-Port Test Metrics:")
print("-" * 50)

for port_id, port_name in TARGET_PORTS.items():
    port_mask = test_df['portid'] == port_id
    if port_mask.sum() > 0:
        port_y_test = y_test[port_mask]
        port_y_pred = y_test_pred[port_mask]
        
        mae = mean_absolute_error(port_y_test, port_y_pred)
        print(f"  {port_name}: MAE = {mae:.2f} days")

In [ ]:
# Plot actual vs predicted
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

for ax, (y_true, y_pred, name) in zip(axes, 
    [(y_train, y_train_pred, 'Train'), (y_val, y_val_pred, 'Validation'), (y_test, y_test_pred, 'Test')]):
    ax.scatter(y_true, y_pred, alpha=0.3, s=5)
    ax.plot([0, 15], [0, 15], 'r--', label='Perfect')
    ax.set_xlabel('Actual Delay (days)')
    ax.set_ylabel('Predicted Delay (days)')
    ax.set_title(f'{name} Set')
    ax.set_xlim(0, 15)
    ax.set_ylim(0, 15)
    ax.legend()

plt.tight_layout()
plt.show()

## 7. Feature Importance Analysis

In [ ]:
# Feature importance
importance_df = pd.DataFrame({
    'feature': available_features,
    'importance': model.feature_importance(importance_type='gain')
}).sort_values('importance', ascending=False)

print("Top 15 Features by Importance:")
print(importance_df.head(15).to_string(index=False))

In [ ]:
# Plot feature importance
plt.figure(figsize=(10, 8))
plt.barh(importance_df['feature'].head(15), importance_df['importance'].head(15))
plt.xlabel('Feature Importance (Gain)')
plt.title('Top 15 Most Important Features')
plt.gca().invert_yaxis()
plt.tight_layout()
plt.show()

In [ ]:
# SHAP analysis (optional - requires shap package)
try:
    import shap
    
    # Sample for SHAP (too slow on full dataset)
    sample_size = min(1000, len(X_test))
    X_sample = X_test.sample(sample_size, random_state=42)
    
    explainer = shap.TreeExplainer(model)
    shap_values = explainer.shap_values(X_sample)
    
    plt.figure(figsize=(12, 8))
    shap.summary_plot(shap_values, X_sample, show=False)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("SHAP not installed. Install with: pip install shap")

## 8. Model Export

In [ ]:
# Save model
import os
os.makedirs('ml_models/saved_models', exist_ok=True)

model_path = 'ml_models/saved_models/port_delay_v1.joblib'
joblib.dump(model, model_path)
print(f"Model saved to: {model_path}")

# Verify loading
loaded_model = joblib.load(model_path)
test_pred = loaded_model.predict(X_test[:5])
print(f"\nTest predictions after reload: {test_pred}")

In [ ]:
# Save feature list for reference
import json

model_info = {
    'model_version': 'v1',
    'training_date': datetime.now().isoformat(),
    'features': available_features,
    'target_ports': TARGET_PORTS,
    'test_metrics': {
        'mae': float(test_metrics['mae']),
        'rmse': float(test_metrics['rmse']),
        'within_1_day_pct': float(test_metrics['within_1_day']),
        'within_2_days_pct': float(test_metrics['within_2_days']),
    }
}

with open('ml_models/saved_models/model_info.json', 'w') as f:
    json.dump(model_info, f, indent=2)

print("Model info saved!")

## 9. Test Integration with FreightCalculator

In [ ]:
# Test the predictor class
from ml_models.port_congestion_predictor import PortCongestionPredictor

predictor = PortCongestionPredictor(
    model_path='ml_models/saved_models/port_delay_v1.joblib',
    data_path='Daily_Port_Activity_Data_and_Trade_Estimates.csv',
    port_database_path='PortWatch_ports_database.csv'
)

print(f"Model available: {predictor.is_model_available()}")

In [ ]:
# Test predictions
test_ports = ['Qingdao', 'Rizhao', 'Mundra', 'Vizag']
test_date = '2026-03-15'

print(f"Predictions for {test_date}:")
print("-" * 60)

for port in test_ports:
    result = predictor.predict(port, test_date)
    print(f"  {port}: {result.predicted_delay_days:.1f} days "
          f"[{result.confidence_lower:.1f} - {result.confidence_upper:.1f}] "
          f"({result.congestion_level}) "
          f"[{result.model_used}]")

In [ ]:
# Test voyage delay function
delay = predictor.get_delay_for_voyage(
    discharge_port='Qingdao',
    eta_date='2026-03-15',
    cargo_quantity=170000
)

print(f"\nVoyage delay for Qingdao on 2026-03-15: {delay:.1f} days")

## Summary

The port congestion prediction model has been trained and saved. Key outputs:

1. **Model**: `ml_models/saved_models/port_delay_v1.joblib`
2. **Model Info**: `ml_models/saved_models/model_info.json`

**Usage in FreightCalculator**:
```python
from ml_models import PortCongestionPredictor

predictor = PortCongestionPredictor('ml_models/saved_models/port_delay_v1.joblib')
delay = predictor.get_delay_for_voyage("Qingdao", "2026-03-15")
result = calculator.calculate_voyage(vessel, cargo, extra_port_delay_days=delay)
```